In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import spearmanr

In [40]:
# Предобработка датасета с данными поступивших в 2023
df_ege_raw = pd.read_excel("БаллЗачисл2023.xlsx", usecols=['L_ID', 'Б.1', 'Б.1.Д', 'Б.1.И', 'Б.2', 'Б.2.Д', 'Б.2.И', 'Б.3', 'Б.3.Д', 'Б.3.И', 'ИД'])
df_ege_raw = df_ege_raw[~(df_ege_raw['Б.1'].isin(['н/с', 'н/р'])) & (df_ege_raw['Б.1'].notna())]
df_ege_raw = df_ege_raw[~(df_ege_raw['Б.2'].isin(['н/с', 'н/р'])) & (df_ege_raw['Б.1'].notna())]
df_ege_raw = df_ege_raw[~(df_ege_raw['Б.3'].isin(['н/с', 'н/р'])) & (df_ege_raw['Б.1'].notna())]
df_ege_raw['Б.1'] = df_ege_raw['Б.1'].astype(str).astype(int)
df_ege_raw['Б.2'] = df_ege_raw['Б.2'].astype(str).astype(int)
df_ege_raw['Б.3'] = df_ege_raw['Б.3'].astype(float).astype(int)
df_ege_raw['ИД'] = df_ege_raw['ИД'].fillna(0)
df_ege_raw['ИД'] = df_ege_raw['ИД'].astype(float).astype(int)
df_ege_raw['Год'] = 2023
df_ege_raw['Год'] = df_ege_raw['Год'].astype(int)
df_ege_raw['L_ID'] = df_ege_raw['L_ID'].astype(int)
df_ege_raw = df_ege_raw.reindex(columns=['L_ID', 'Б.1', 'Б.1.Д', 'Б.1.И', 'Б.2', 'Б.2.Д', 'Б.2.И', 'Б.3', 'Б.3.Д', 'Б.3.И', 'ИД', 'Год'])
df_ege_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908 entries, 0 to 1918
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   L_ID    1908 non-null   int32 
 1   Б.1     1908 non-null   int32 
 2   Б.1.Д   1908 non-null   object
 3   Б.1.И   1908 non-null   object
 4   Б.2     1908 non-null   int32 
 5   Б.2.Д   1908 non-null   object
 6   Б.2.И   1908 non-null   object
 7   Б.3     1908 non-null   int32 
 8   Б.3.Д   1908 non-null   object
 9   Б.3.И   1908 non-null   object
 10  ИД      1908 non-null   int32 
 11  Год     1908 non-null   int32 
dtypes: int32(6), object(6)
memory usage: 149.1+ KB


In [54]:
# Предобработка датасета с данными поступивших в 2019-2022
raw_df_2019_2022 = pd.read_excel('data1.xlsx', usecols=['L_ID', 'B1', 'B1_D', 'B1_T', 'B2', 'B2_D', 'B2_T', 'B3', 'B3_D', 'B3_T', 'ИД', 'Год'])
raw_df_2019_2022 = raw_df_2019_2022.reindex(columns=['L_ID', 'B1', 'B1_D', 'B1_T', 'B2', 'B2_D', 'B2_T', 'B3', 'B3_D', 'B3_T', 'ИД', 'Год'])
raw_df_2019_2022['ИД'] = raw_df_2019_2022['ИД'].fillna(0)
raw_df_2019_2022[['L_ID', 'B1', 'B2', 'B3','ИД', 'Год']] = raw_df_2019_2022[['L_ID', 'B1', 'B2', 'B3','ИД', 'Год']].astype('int')
raw_df_2019_2022[['B1_D', 'B1_T', 'B2_D', 'B2_T', 'B3_D', 'B3_T']] = raw_df_2019_2022[['B1_D', 'B1_T', 'B2_D', 'B2_T', 'B3_D', 'B3_T']].astype('str')
raw_df_2019_2022.columns = ['L_ID', 'Б.1', 'Б.1.Д', 'Б.1.И', 'Б.2', 'Б.2.Д', 'Б.2.И', 'Б.3', 'Б.3.Д', 'Б.3.И', 'ИД', 'Год']
raw_df_2019_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7299 entries, 0 to 7298
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   L_ID    7299 non-null   int32 
 1   Б.1     7299 non-null   int32 
 2   Б.1.Д   7299 non-null   object
 3   Б.1.И   7299 non-null   object
 4   Б.2     7299 non-null   int32 
 5   Б.2.Д   7299 non-null   object
 6   Б.2.И   7299 non-null   object
 7   Б.3     7299 non-null   int32 
 8   Б.3.Д   7299 non-null   object
 9   Б.3.И   7299 non-null   object
 10  ИД      7299 non-null   int32 
 11  Год     7299 non-null   int32 
dtypes: int32(6), object(6)
memory usage: 513.3+ KB


In [65]:
#UNION 2019-2022 и 2023
df_ege_raw_2019_23 = pd.concat([df_ege_raw, raw_df_2019_2022], ignore_index=True)
df_ege_raw_2019_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9207 entries, 0 to 9206
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   L_ID    9207 non-null   int32 
 1   Б.1     9207 non-null   int32 
 2   Б.1.Д   9207 non-null   object
 3   Б.1.И   9207 non-null   object
 4   Б.2     9207 non-null   int32 
 5   Б.2.Д   9207 non-null   object
 6   Б.2.И   9207 non-null   object
 7   Б.3     9207 non-null   int32 
 8   Б.3.Д   9207 non-null   object
 9   Б.3.И   9207 non-null   object
 10  ИД      9207 non-null   int32 
 11  Год     9207 non-null   int32 
dtypes: int32(6), object(6)
memory usage: 647.5+ KB
